## TF-IDF dataset EDA

In [ ]:
tfidf = DataFrame(CSV.File(tfidf_csv_path))

println(size(tfidf))
println(names(tfidf))
println(describe(tfidf))

using JSON
tfidf_keywords = tfidf[!, :Keywords]
tfidf_keywords = [JSON.parse(replace(tfidf_keywords[i], "'" => "\""), dicttype=Dict{String,Float64}) for i in 1:length(tfidf_keywords)]

tfidf[!, :Count] = [length(tfidf_keywords[i]) for i in 1:length(tfidf_keywords)]
sort!(tfidf, [:Count], rev=true)


start_date = Date.("2022-02-01", "yyyy-mm-dd")
end_date = Date.(Dates.now())

p = plot(
        tfidf[!, :Date],
        tfidf[!, :Count],
        seriestype=:scatter,
        xlabel="Date",
        ylabel="Count",
        xlims=Dates.value.([start_date, end_date]),
        title="Date vs Unique words count",
        legend=true,
        label="count",
        grid=false,
        size=(1200, 800),
);

xticks!(p, Dates.value.([start_date:Dates.Month(1):end_date;]), Dates.format.([start_date:Dates.Month(1):end_date;], "yyyy-mm"))

p

In [ ]:
wordFrequencyToDateFrame = DataFrame()
# word_collection = Dict{String,Int64}()

file_contents = ""
for (root, dirs, files) in walkdir("../0_data_scrapping/results/isw/")
    for file in files
        if endswith(file, ".txt")
            file_path = joinpath(root, file)
            file_content = read(file_path, String)
            file_contents = file_contents * " " * file_content

            words_count = length(split(file_content))
            date = replace(file, "assessment-" => "")
            date = replace(date, ".txt" => "")
            date = Date.(date, "yyyy-mm-dd")
            wordFrequencyToDateFrame = vcat(wordFrequencyToDateFrame, DataFrame(Date=date, Count=words_count))
        end
    end
end

# call do_processing() function in 2_data_prepartion/text_preprocessing.py using pycall

# println(PyCall.python)

# println(joinpath(@__DIR__, "..", "2_data_preparation"))



using PyCall
pushfirst!(PyVector(pyimport("sys")["path"]), joinpath(@__DIR__, "..", "1_data_preparation"))
text_processing = pyimport("text_preprocessing")
file_contents = text_processing.do_preprocessing(file_contents)



wordFrequency = countmap(file_contents)
wordFrequencyToDateFrame = DataFrame(wordFrequency)
# set name for dataframe
rename!(wordFrequencyToDateFrame)
# wordFrequencyToDateFrame
describe(wordFrequencyToDateFrame)



using OrderedCollections

od = OrderedDict(wordFrequency)
od_vector = sort!(od; byvalue=true, rev=true)


od_vector = first(od_vector, 100)
od_vector = map(x -> [x.first, string(x.second)], od_vector)

words_for_table = map(x -> x[1], od_vector)
counts_for_table = map(x -> x[2], od_vector)

Base.displaysize(x::DataFrame) = (100, 100)
dadasdasd = DataFrame(Word=words_for_table, Count=counts_for_table)
dadasdasd.Count = map(x -> parse(Int64, x), dadasdasd.Count)
show(dadasdasd, allrows=true)

In [ ]:
# convert dadasdasd.Count to Int64

# visualize dadasdasd
using Printf

bar(
    dadasdasd.Word,
    dadasdasd.Count,
    xlabel="Word",
    ylabel="Count",
    title="Word frequency",
    label="Number of occurances",
    xrotation=90,
    bar_width=0.3,
    xticks=:all,
    yticks=0:1000:50000,
    yrotation=0,
    yformatter=y -> @sprintf("%d", y),
    aspect_ratio=:none,
    size=(1500, 1000),
)